In [ ]:
import sys
sys.path.append("c:/Program Files/Stata18/utilities") #Importamos al ubicacion de Stata en el sistema

#cargamos stata y la configuración multiplataforma
from pystata import config
config.init('mp')

# 1. ESTADÍSTICA DESCRIPTIVA FINAL

### 1.1. Configuración variables y carga del modelo predictivo

In [ ]:
%%stata
use "EPA1819.dta", clear

*Eliminamos de la muestra a inactivos jubilados e incapacitados
label define pobinc 0 "Ns/Nc" 1 "Estudiante" 2 "Jubilado" 3 "Labores Hogar" ///
			4 "Incapacitado" 5 "Pension publica" 6 "Voluntario" 7 "Otros"
label values sidi1 pobinc

**Período de la encuesta
***Año
gen anoenc=.
replace anoenc=0 if ciclo>=182 & ciclo<=185
replace anoenc=1 if ciclo>=186 & ciclo<=189
label define ae 0 "2018" 1 "2019"
label values anoenc ae

***Trimestre
gen trenc=ciclo
label define tr 182 "2018T1" 183 "2018T2" 184 "2018T3" 185 "2018T4" ///
                         186 "2019T1" 187 "2019T2" 188 "2019T3" 189 "2019T4"
label values trenc tr

**Sexo
rename sexo1 hombre
replace hombre=0 if hombre==6

**Edad
rename edad5 edad
drop if edad<=10
drop if edad>=65
gen edad5 = .
replace edad5 = 1 if edad == 16
replace edad5 = 2 if edad == 20
replace edad5 = 3 if edad == 25
replace edad5 = 4 if edad == 30
replace edad5 = 5 if edad == 35
replace edad5 = 6 if edad == 40
replace edad5 = 7 if edad == 45
replace edad5 = 8 if edad == 50
replace edad5 = 9 if edad == 55
replace edad5 = 10 if edad == 60 
label define e5 1 "16 a 19 anos" 2 "20 a 24 anos" 3 "25 a 29 anos" 4 "30 a 34 anos" 5 "35 a 39 anos" 6 "40 a 44 anos" 7 "45 a 49 anos" 8 "50 a 54 anos" 9 "55 a 59 anos" 10 "60 a 64 anos"
label values edad5 e5

**Provincia
label define prov_lbl 1 "Araba/Álava" 2 "Albacete" 3 "Alicante/Alacant" 4 "Almería" 5 "Ávila" ///
    6 "Badajoz" 7 "Balears, Illes" 8 "Barcelona" 9 "Burgos" 10 "Cáceres" 11 "Cádiz" 12 "Castellón /Castelló" ///
    13 "Ciudad Real" 14 "Córdoba" 15 "Coruña, A" 16 "Cuenca" 17 "Girona" 18 "Granada" 19 "Guadalajara" ///
    20 "Gipuzkoa" 21 "Huelva" 22 "Huesca" 23 "Jaén" 24 "León" 25 "Lleida" 26 "Rioja, La" 27 "Lugo" ///
    28 "Madrid" 29 "Málaga" 30 "Murcia" 31 "Navarra" 32 "Ourense" 33 "Asturias" 34 "Palencia" ///
    35 "Palmas, Las" 36 "Pontevedra" 37 "Salamanca" 38 "Santa Cruz de Tenerife" 39 "Cantabria" 40 "Segovia" ///
    41 "Sevilla" 42 "Soria" 43 "Tarragona" 44 "Teruel" 45 "Toledo" 46 "Valencia/València" 47 "Valladolid" ///
    48 "Bizkaia" 49 "Zamora" 50 "Zaragoza" 51 "Ceuta" 52 "Melilla"
label values prov prov_lbl

**Variable pais de procedencia
gen paisnac = 1
replace paisnac = 2 if regna1==115
replace paisnac = 3 if inlist(regna1, 125, 127)
replace paisnac = 4 if inlist(regna1, 128, 100)
replace paisnac = 5 if inlist(regna1, 200)
replace paisnac = 6 if inlist(regna1, 300)
replace paisnac = 7 if inlist(regna1, 310, 350)
replace paisnac = 8 if inlist(regna1, 400, 410, 420, 500)
label define pais 1 "Espana" 2 "UE-15" 3 "UE-27" 4 "Resto de Europa" 5 "Africa" 6 "America del Norte" 7 "Centroamerica y Sudamerica" 8 "Asia y Oceania"
label values paisnac pais

**Extranjero
gen extranjero=0
replace extranjero=1 if paisnac!=1 

**Nivel educativo
gen educ=.
replace educ=1 if nforma == "AN"
replace educ=2 if nforma == "P1"
replace educ=3 if nforma == "P2"
replace educ=4 if nforma == "S1"
replace educ=5 if nforma == "SG" | nforma == "SP"
replace educ=6 if nforma == "SU"
label define nivest  1 "Sin Estudios" 2 "Primaria Incompleta" 3 "Primaria" 4 "Secundaria" 5 "FP Media, Bachillerato" 6 "Educación Superior (FP, Univ.)"
label values educ nivest

**Tiempo en meses en la empresa
rename dcom relab
replace relab=0  if aoi>=5

**VARIABLES DEPENDIENTES
***AOI: Situación Laboral
label define tt 3 "Ocupados Subempleados" 4 "Resto Ocupados" 5 "Parados sin exp." 6 "Parados con exp." 7 "Inactivos desanimados" 8 "Inactivos pontenciales act." 9 "Resto Inactivos"
label values aoi tt

***SIDI1: Situación inactividad
label define inact 0 "ns/nc" 1 "Estudiante" 2 "Jubilado" 3 "Labores del hogar" 4 "Incapacitado" 5 "Pensionista no contributivo" 6 "Trabajo sociales sin remuneración" 7 "Otros"
label values sidi1 inact

***ACTIVIDAD
gen activo=.
replace activo=1 if inlist(aoi, 3, 4, 5, 6)
replace activo=0 if inlist(aoi, 7, 8, 9) & inlist(sidi1, 1, 2 ,3, 5) //Inactivos (aoi), Potencialemente activos o grupos de interes (jubilado) (sidi1)

***EMPLEO
gen empleado=.
replace empleado=1 if inlist(aoi, 3, 4)
replace empleado=0 if inlist(aoi, 5, 6)

*Homogeneizamos variables utilizadas
**Edad
qui tab edad5, gen(ed)
global edad ed1 ed2 ed3 ed4 ed5 ed6 ed7 ed8 ed9
**Paisnac
qui tab paisnac, gen(pais)
global paisnac pais2 pais3 pais4 pais5 pais6 pais7 pais8
**Educación
qui tab educ, gen(ned)
global educ ned1 ned2 ned3 ned4 ned5

*************************Importamos el mejor modelo*****************************
est use 2.cvprov.ster
predict ycvprov
qui sum ycvprov, detail
scalar p5cvprov=r(p5)

*Creamos variable afectado por la reforma
gen tratado=.
replace tratado=1 if ycvprov>=0.150863595 //0.01 puntos mas de margen sobre valor hallado en 3.CutOffPrv.do
replace tratado=0 if ycvprov<0.130863595 & ycvprov>p5cvprov //0.01 puntos mas de margen sobre valor hallado en 3.CutOffPrv.do

*Creamos la variable de diferencia en diferencias
gen did=.
replace did=tratado*anoenc

### 1.2. Distribución predicciones

In [ ]:
%%stata

*Histograma por modelo
foreach ef in prov {
	foreach mod in bic cv {
		histogram y`mod'`ef' if y`mod'`ef'>0.0025, /// Eliminamos prob < 0,25%
		xline(0.1409) /// Probabilidad media de verse afectado
		title(Modelo `mod' con `ef') ///
		xtitle("Prob. verse afectado por subida") ///
		xlabel(0 "0%" 0.1409 "14,09%" 0.3 "30%" 0.5 "50%" 0.75 "75%" 1 "100%") ///
		ytitle("Número de personas (Miles)") ///
		ylabel(5.0e+04 "50" 1.0e+05 "100" 1.5e+05 "150") ///
		frequency bin(30)
		graph save 1.Pred`mod'`ef'.gph, replace
	}
}

**Guardamos resumen de los modelos
graph combine 1.Predbicprov.gph 1.Predcvprov.gph, ///
	title({bf:GRÁFICO 4: VALORES PREDICHOS (EPA)})
graph export "1.PredResumen.png", width(1350) height(700) as(png) replace

### 1.3. Tabla comparativa

In [ ]:
%%stata

**Dummies y continua
foreach x in hombre extranjero relab {
	qui ttest `x', by(tratado)
	scalar med`x'0=r(mu_1) //No tratado
	scalar sd`x'0=r(sd_1) 
	scalar med`x'1=r(mu_2) //Tratado
	scalar sd`x'1=r(sd_2) 
	scalar `x'diff=r(mu_2)-r(mu_1) //Diferencia de medias
	scalar `x'pvalue=r(p)
	matrix `x'= (med`x'1, med`x'0, `x'diff \ sd`x'1, sd`x'0, `x'pvalue)
}

matrix dumm=(hombre \ extranjero \ relab) 
matrix colnames dumm= "Tratado (Media/ Desv. Est.)" ///
						"No tratado (Media/ Desv. Est.)" ///
						"Diferencia (Media/ P-valor)"
matrix rownames dumm= "Hombre" "" ///
						"Extranjero" "" ///
						"Antiguedad en la empresa" ""			
						
**Edad
qui tab edad5, gen(gedad) //Creamos grupos de edad
foreach x in 1 2 3 4 5 6 7 8 9 10{
	qui ttest gedad`x', by(tratado)
	scalar medgedad`x'0=r(mu_1) //No tratado
	scalar sdgedad`x'0=r(sd_1)
	scalar medgedad`x'1=r(mu_2) //Tratado
	scalar sdgedad`x'1=r(sd_2)
	scalar gedad`x'diff=r(mu_2)-r(mu_1) //Diferencia de medias
	scalar gedad`x'pvalue=r(p)
	matrix edad`x'= (medgedad`x'1, medgedad`x'0, gedad`x'diff \ ///
						sdgedad`x'1, sdgedad`x'0, gedad`x'pvalue)
}

matrix edad=(edad1 \ edad2 \ edad3 \ edad4 \ edad5 \ edad6 \ edad7 \ edad8 \ edad9 \ edad10)
matrix colnames edad= "Tratado (Media/ Desv. Est.)" ///
						"No tratado (Media/ Desv. Est.)" ///
						"Diferencia (Media/ P-valor)"
matrix rownames edad = "16-19 años" "-" ///
                       "20-24: años" "-" ///
                       "25-29: años" "-" ///
                       "30-34: años" "-" ///
                       "35-39: años" "-" ///
                       "40-44: años" "-" ///
                       "45-49: años" "-" ///
                       "50-54: años" "-" ///
                       "55-59: años" "-" ///
                       "60-64: años" "-"
					   
**Nivel educativo
qui tab educ, gen(educ) //Creamos grupos
foreach x in 1 2 3 4 5 6 {
	qui ttest educ`x', by(tratado)
	scalar mededuc`x'0=r(mu_1) //No tratado
	scalar sdeduc`x'0=r(sd_1)
	scalar mededuc`x'1=r(mu_2) //Tratado
	scalar sdeduc`x'1=r(sd_2)
	scalar educ`x'diff=r(mu_2)-r(mu_1) //Diferencia de medias
	scalar educ`x'pvalue=r(p)
	matrix educ`x'= (mededuc`x'1, mededuc`x'0, educ`x'diff \ ///
						sdeduc`x'1, sdeduc`x'0, educ`x'pvalue)
}

matrix educ=(educ1 \ educ2 \ educ3 \ educ4 \ educ5 \ educ6)
matrix colnames educ= "Tratado (Media/ Desv. Est.)" ///
						"No tratado (Media/ Desv. Est.)" ///
						"Diferencia (Media/ P-valor)"
matrix rownames educ = "Sin estudios" "-" ///
                       "Primaria Incompleta" "-" ///
                       "Primaria" "-" ///
                       "Secundaria" "-" ///
                       "FP media, Bachillerato" "-" ///
                       "Educación superior" "-"

**Exportamos Resultados
***Tratados y no tratados
qui count if tratado==0 | tratado==1
scalar N=r(N)
foreach x in 1 0 { 
	qui count if tratado==`x'
	scalar n`x'=r(N)
	scalar pct`x'=n`x'/N
}
matrix sum =(n1, n0, 0 \ pct1, pct0, 0)
matrix colnames sum= "Tratado (Media/ Desv. Est.)" ///
						"No tratado (Media/ Desv. Est.)" ///
						"Diferencia (Media/ P-valor)"
matrix rownames sum= "Obs" "Pct"

**Unimos matrices
matrix estdes=(dumm \ edad \ educ \ sum)
putexcel set 1.EstDes.xlsx, replace
putexcel A1=matrix(estdes), nformat(number_d2) names

# 2. RESULTADOS AGREGADOS

In [ ]:
%%stata

*Probabilidad de  pertenecer al mercado laboral
**Creamos la global
global a anoenc tratado did i.prov
global b hombre extranjero i.edad5 i.educ

*Estimación DiD (actividad)
**Sin Xi
qui reg activo $a if activo!=. & tratado!=., robust
estadd local Reg "(Prov)"
estadd local Xi "(No)"
qui sum activo
estadd scalar Promedio=r(mean)
estadd scalar R2=e(r2)
estadd scalar Nobs=e(N)
est store modelo11
**Con Xi
qui reg activo $a $b if activo!=. & tratado!=., robust
estadd local Reg "(Prov)"
estadd local Xi "(Si)"
qui sum activo
estadd scalar Promedio=r(mean)
estadd scalar R2=e(r2)
estadd scalar Nobs=e(N)
est store modelo12

*Probabilidad de estar empleado
**Sin Xi
qui reg empleado $a if empleado!=. & tratado!=., robust
estadd local Reg "(Prov)"
estadd local Xi "(No)"
qui sum empleado
estadd scalar Promedio=r(mean)
estadd scalar R2=e(r2)
estadd scalar Nobs=e(N)
est store modelo13
**Con Xi
qui reg empleado $a $b if empleado!=. & tratado!=., robust
estadd local Reg "(Prov)"
estadd local Xi "(Si)"
qui sum empleado
estadd scalar Promedio=r(mean)
estadd scalar R2=e(r2)
estadd scalar Nobs=e(N)
est store modelo14


*Exportamos resultados
**Actividad
esttab modelo11 modelo12 modelo21 modelo22 using 2.RegAct.rtf, keep(anoenc tratado did) se ///
	title("Regresion 1: Participacion mercado laboral") ///
	mtitles("Lasso BIC" "Lasso BIC" "Lasso CV" "Lasso CV") ///
	s(Reg Xi Promedio R2 Nobs) ///
	coeflabels(anoenc "Diferencia" tratado "Tratado" did "DiD") ///
	star(* 0.1 ** 0.05 *** 0.01) replace
	
**Empleo
esttab modelo13 modelo14 modelo23 modelo24 using 2.RegEmpl.rtf, keep(anoenc tratado did) se ///
	title("Regresion 2: Participación empleo") ///
	mtitles("Lasso BIC" "Lasso BIC" "Lasso CV" "Lasso CV") ///
	s(Reg Xi Promedio R2 Nobs) ///
	coeflabels(anoenc "Diferencia" tratado "Tratado" did "DiD") ///
	star(* 0.1 ** 0.05 *** 0.01) replace

# 3. HETEROGENEIDAD RESULTADOS

### 3.1. Resultados por edad

In [ ]:
%%stata

**Grupos de edad
gen adult=.
replace adult=1 if inlist(edad5, 1,2,3,4,5,6,7) //Hasta 50 años
replace adult=2 if inlist(edad5, 8,9,10) //Más de 50 años
label define adul 1 "Adulto Joven y medio" 2 "Adulto Mayor"
label value adult adul

**Estimaciones
foreach n in 1 2 {
	preserve
	*Filtramos la muestra por edad
	keep if adult==`n'
	**Afectado por año
	replace did=.
	replace did=tratado*anoenc
	**Creamos la global
	global a anoenc tratado did i.prov
	global b hombre extranjero i.educ

	**Estimamos para actividad
	qui reg activo $a $b if activo!=. & tratado!=., robust
	estadd local Reg "(Prov)"
	estadd local Xi "(Si)"
	qui sum activo
	estadd scalar Promedio=r(mean)
	estadd scalar R2=e(r2)
	estadd scalar Nobs=e(N)
	est store edad_act`n'

	**Estimamos para empleo
	qui reg empleado $a $b if activo!=. & tratado!=., robust
	estadd local Reg "(Prov)"
	estadd local Xi "(Si)"
	qui sum empleado
	estadd scalar Promedio=r(mean)
	estadd scalar R2=e(r2)
	estadd scalar Nobs=e(N)
	est store edad_empl`n'
	restore
}

***********************************Resultados***********************************
*Actividad y empleo
esttab edad_act1 edad_act2 edad_empl1 edad_empl2 using 2.RegEdad.rtf, keep(anoenc tratado did) ///
	title("Regresion: Resultados edad") ///
	mtitles("Menores 50 años" "Mayores 50 años" "Menores 50 años" "Mayores 50 años") ///
	s(Reg Xi Promedio R2 Nobs) ///
	coeflabels(anoenc "Diferencia" tratado "Tratado" did "DiD") ///
	star(* 0.1 ** 0.05 *** 0.01) replace

### 3.2. Resultados por género

In [ ]:
%%stata
foreach n in 0 1 {
	preserve
	*Filtramos la muestra por sexo
	keep if hombre==`n'
	**Afectado por año
	replace did=.
	replace did=tratado*anoenc
	**Creamos la global
	global a anoenc tratado did i.prov
	global b extranjero i.edad5 i.educ

	**Estimamos para actividad
	qui reg activo $a $b if activo!=. & tratado!=., robust
	estadd local Reg "(Prov)"
	estadd local Xi "(Si)"
	qui sum activo
	estadd scalar Promedio=r(mean)
	estadd scalar R2=e(r2)
	estadd scalar Nobs=e(N)
	est store gen_act`n'

	**Estimamos para empleo
	qui reg empleado $a $b if activo!=. & tratado!=., robust
	estadd local Reg "(Prov)"
	estadd local Xi "(Si)"
	qui sum empleado
	estadd scalar Promedio=r(mean)
	estadd scalar R2=e(r2)
	estadd scalar Nobs=e(N)
	est store gen_empl`n'
	restore
}

***********************************Resultados***********************************
*Actividad y empleo
esttab gen_act0 gen_act1 gen_empl0 gen_empl1 using 2.RegSexo.rtf, keep(anoenc tratado did) ///
	title("Regresión: Resultado sexo") ///
	mtitles("Mujeres" "Hombres" "Mujeres" "Hombres") ///
	s(Reg Xi Promedio R2 Nobs) ///
	coeflabels(anoenc "Diferencia" tratado "Tratado" did "DiD") ///
	star(* 0.1 ** 0.05 *** 0.01) replace

### 3.3. Resultados por nivel educativo

In [ ]:
%%stata
**Grupos de educación
gen neduc=.
replace neduc=1 if inlist(educ, 1,2,3,4)
replace neduc=2 if inlist(educ, 5, 6)
label define ned_lb 1 "Eduación Obligatoria" 2 "Educación Posobligatoria"
label values neduc ned_lb

foreach n in 1 2 {
	preserve
	*Filtramos la muestra por nivel educativo
	keep if neduc==`n'
	**Afectado por año
	replace did=.
	replace did=tratado*anoenc
	**Creamos la global
	global a anoenc tratado did i.prov
	global b hombre extranjero i.edad5

	**Estimamos para actividad
	qui reg activo $a $b if activo!=. & tratado!=., robust
	estadd local Reg "(Prov)"
	estadd local Xi "(Si)"
	qui sum activo
	estadd scalar Promedio=r(mean)
	estadd scalar R2=e(r2)
	estadd scalar Nobs=e(N)
	est store educ_act`n'

	**Estimamos para empleo
	qui reg empleado $a $b if activo!=. & tratado!=., robust
	estadd local Reg "(Prov)"
	estadd local Xi "(Si)"
	qui sum empleado
	estadd scalar Promedio=r(mean)
	estadd scalar R2=e(r2)
	estadd scalar Nobs=e(N)
	est store educ_empl`n'
	restore
}

***********************************Resultados***********************************
*Actividad
esttab educ_act1 educ_act2 educ_empl1 educ_empl2 using 2.RegEduc.rtf, keep(anoenc tratado did) ///
	title("Regresión: Resultado nivel educativo") ///
	mtitles("Eduación Obligatoria" "Educación Posobligatoria" "Eduación Obligatoria" "Educación Posobligatoria") ///
	s(Reg Xi Promedio R2 Nobs) ///
	coeflabels(anoenc "Diferencia" tratado "Tratado" did "DiD") ///
	star(* 0.1 ** 0.05 *** 0.01) replace